In [1]:
import requests
import pandas as pd
import itertools

데이터 전처리란 획득한 데이터를 필요한 형태로 가공하고 정제하는 작업을 의미합니다. 우리가 수집한 데이터는 몇 가지 문제가 있습니다. 이러한 문제를 해결하기 위해 데이터 전처리를 진행하겠습니다.

1. 동명이인인 국회의원이 존재합니다.
2. 필요 없는 데이터 칼럼이 다수 존재합니다.
3. 네트워크 분석을 위해서는 국회의원을 node 리스트로 만들어야 합니다.
4. 네트워크 분석을 위해서는 공동 발의에 참여한 국회의원들 간의 관계를 나타내는 edge 리스트가 필요합니다.


## 1. 동명이인 국회의원 확인 및 처리

- 제21대: 더불어민주당 김병욱 의원과 미래통합당 김병욱 의원, 더불어민주당 이수진 의원과 더불어시민당(추후 합당됨) 이수진 의원
  - 더불어민주당 김병욱 의원: 김병욱A
  - 미래통합당 김병욱 의원: 김병욱B
  - 더불어민주당 이수진 의원: 이수진A
  - 더불어시민당 이수진 의원: 이수진B
- 제20대: 새누리당 최경환 의원과 국민의당 최경환 의원, 새누리당 김성태 의원과 새누리당 (비례) 김성태 의원
  - 새누리당 최경환 의원: 최경환A
  - 국민의당 최경환 의원: 최경환B
  - 새누리당 김성태 의원: 김성태A
  - 새누리당 (비례) 김성태 의원: 김성태B


In [2]:
# 데이터 불러오기
lawmaker_20 = pd.read_csv("./data/lawmaker_20.csv")
lawmaker_21 = pd.read_csv("./data/lawmaker_21.csv")
laws_20 = pd.read_csv("./data/laws_20.csv")
laws_21 = pd.read_csv("./data/laws_21.csv")

In [3]:
laws_20["PUBL_PROPOSER"]

0           고용진,김병욱,김철민,박재호,박정,서형수,심기준,위성곤,유승희,이규희,이원욱,홍익표
1                      김철민,박재호,서형수,심기준,위성곤,유승희,이규희,이원욱,이종걸
2                      김관영,김동철,김종훈,박선숙,박찬대,이용득,이철희,정인화,추혜선
3                  강석호,강효상,김규환,김명연,김석기,김용태,민경욱,이철규,정갑윤,정운천
4                  김석기,김정재,성일종,송언석,송희경,유동수,윤종필,이양수,이종배,함진규
                               ...                        
21589                  김경협,백재현,변재일,안규백,오제세,원혜영,이춘석,정성호,조정식
21590              권성동,김재경,박덕흠,박명재,배덕광,안상수,이철우,이학재,이현재,주호영
21591              김경협,김현미,민병두,박홍근,변재일,안규백,오제세,원혜영,이춘석,정성호
21592    김세연,김정훈,김한표,박맹우,박주선,유기준,유재중,이종배,이진복,이채익,이헌승,조경...
21593    김경협,노웅래,박광온,박영선,박지원,박찬대,소병훈,송영길,신경민,오제세,윤후덕,이찬...
Name: PUBL_PROPOSER, Length: 21594, dtype: object

In [4]:
def disambiguate20(x):
    if pd.isnull(x["PUBL_PROPOSER"]):
        proposers = []
    else:
        proposers = x["PUBL_PROPOSER"].split(",")
    repr_proposer = x["RST_PROPOSER"]
    if "," in repr_proposer:
        repr_proposer = repr_proposer.split(",")
    else:
        repr_proposer = [repr_proposer]
    proposers_list = x["MEMBER_LIST"]
    proposers.extend(repr_proposer)

    if "최경환" not in proposers and "김성태" not in proposers:
        x["proposers"] = proposers
        x["repr_proposer"] = repr_proposer
        return x
    else:
        # 최경환이 있는 경우
        if "최경환" in proposers:
            page = requests.get(proposers_list).text
            a = "崔炅煥"  # 새누리당 최경환
            b = "崔敬煥"  # 국민의당 최경환
            if a in page:
                proposers[proposers.index("최경환")] = "최경환A"
            elif b in page:
                proposers[proposers.index("최경환")] = "최경환B"
        # 김성태가 있는 경우
        if "김성태" in proposers:
            page = requests.get(proposers_list).text
            a = "金聖泰"  # 새누리당 김성태
            b = "金成泰"  # 새누리당 (비례) 김성태
            if a in page:
                proposers[proposers.index("김성태")] = "김성태A"
            elif b in page:
                proposers[proposers.index("김성태")] = "김성태B"
        x["proposers"] = proposers
        # 대표발의자가 최경환이나 김성태인 경우
        if "A" in proposers[-1] or "B" in proposers[-1]:
            x["repr_proposer"] = [proposers[-1]]
        else:
            x["repr_proposer"] = repr_proposer
        return x


laws_20 = laws_20.apply(disambiguate20, axis="columns")

In [5]:
# lawmaker_20 데이터프레임에서 이름 변경
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "崔炅煥"].index, "HG_NM"] = (
    "최경환A"
)
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "崔敬煥"].index, "HG_NM"] = (
    "최경환B"
)
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "金聖泰"].index, "HG_NM"] = (
    "김성태A"
)
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "金成泰"].index, "HG_NM"] = (
    "김성태B"
)

In [6]:
def disambiguate21(x):
    if pd.isnull(x["PUBL_PROPOSER"]):
        proposers = []
    else:
        proposers = x["PUBL_PROPOSER"].split(",")
    repr_proposer = x["RST_PROPOSER"]
    if "," in repr_proposer:
        repr_proposer = repr_proposer.split(",")
    else:
        repr_proposer = [repr_proposer]
    proposers_list = x["MEMBER_LIST"]
    proposers.extend(repr_proposer)

    if "김병욱" not in proposers and "이수진" not in proposers:
        x["proposers"] = proposers
        x["repr_proposer"] = repr_proposer
        return x
    else:
        # 김병욱이 있는 경우. 둘은 한자 까지 같다...
        if "김병욱" in proposers:
            page = requests.get(proposers_list).text
            a = "김병욱(더불어민주당/金炳旭)"  # 더불어민주당 김병욱
            if a in page:
                proposers[proposers.index("김병욱")] = "김병욱A"
            else:  # 국민의힘은 정당명이 자주 바뀌었기 때문에 else 처리한다.
                proposers[proposers.index("김병욱")] = "김병욱B"
        # 이수진이 있는 경우
        if "이수진" in proposers:
            page = requests.get(proposers_list).text
            a = "李秀眞"  # 더불어민주당 이수진
            b = "李壽珍"  # 더불어시민당 이수진
            if a in page:
                proposers[proposers.index("이수진")] = "이수진A"
            elif b in page:
                proposers[proposers.index("이수진")] = "이수진B"
        x["proposers"] = proposers
        # 대표발의자가 김병욱이나 이수진인 경우
        if "A" in proposers[-1] or "B" in proposers[-1]:
            x["repr_proposer"] = [proposers[-1]]
        else:
            x["repr_proposer"] = repr_proposer
        return x


laws_21 = laws_21.apply(disambiguate21, axis="columns")

In [7]:
# lawmaker_21 데이터프레임에서 이름 변경
lawmaker_21.loc[lawmaker_21[lawmaker_21["HJ_NM"] == "李秀眞"].index, "HG_NM"] = (
    "이수진A"
)
lawmaker_21.loc[lawmaker_21[lawmaker_21["HJ_NM"] == "李壽珍"].index, "HG_NM"] = (
    "이수진B"
)
lawmaker_21.loc[
    lawmaker_21[
        (lawmaker_21["HG_NM"] == "김병욱") & (lawmaker_21["POLY_NM"] == "더불어민주당")
    ].index,
    "HG_NM",
] = "김병욱A"
lawmaker_21.loc[
    lawmaker_21[
        (lawmaker_21["HG_NM"] == "김병욱") & (lawmaker_21["POLY_NM"] == "더불어민주당")
    ].index,
    "HG_NM",
] = "김병욱B"

## 2. 필요 없는 칼럼 제거 및 데이터 정제


In [8]:
laws_20.columns

Index(['BILL_ID', 'BILL_NO', 'BILL_NAME', 'COMMITTEE', 'PROPOSE_DT',
       'PROC_RESULT', 'AGE', 'DETAIL_LINK', 'PROPOSER', 'MEMBER_LIST',
       'LAW_PROC_DT', 'LAW_PRESENT_DT', 'LAW_SUBMIT_DT', 'CMT_PROC_RESULT_CD',
       'CMT_PROC_DT', 'CMT_PRESENT_DT', 'COMMITTEE_DT', 'PROC_DT',
       'COMMITTEE_ID', 'PUBL_PROPOSER', 'LAW_PROC_RESULT_CD', 'RST_PROPOSER',
       'proposers', 'repr_proposer'],
      dtype='object')

In [9]:
# 20대 법률 발의안 데이터에서 필요한 칼럼만 추출하여 저장한다.
laws_20_preprocessed = laws_20[
    ["BILL_ID", "BILL_NAME", "MEMBER_LIST", "proposers", "repr_proposer"]
]
laws_20_preprocessed.to_csv("./data/laws_20_preprocessed.csv", index=False)

In [10]:
# 21대도 마찬가지
laws_21_preprocessed = laws_21[
    ["BILL_ID", "BILL_NAME", "MEMBER_LIST", "proposers", "repr_proposer"]
]
laws_21_preprocessed.to_csv("./data/laws_21_preprocessed.csv", index=False)

In [11]:
# lawmaker 테이블도 필요한 칼럼만 추출하여 저장한다.
lawmaker_20["elected_cnt"] = lawmaker_20["UNITS"].str.split(",").apply(len)
lawmaker_20_preprocessed = lawmaker_20[
    [
        "HG_NM",
        "HG_NM",
        "HJ_NM",
        "BTH_DATE",
        "SEX_GBN_NM",
        "elected_cnt",
        "POLY_NM",
        "ORIG_NM",
    ]
]
lawmaker_21["elected_cnt"] = lawmaker_21["UNITS"].str.split(",").apply(len)
lawmaker_21_preprocessed = lawmaker_21[
    [
        "HG_NM",
        "HG_NM",
        "HJ_NM",
        "BTH_DATE",
        "SEX_GBN_NM",
        "elected_cnt",
        "POLY_NM",
        "ORIG_NM",
    ]
]
lawmaker_20_preprocessed.to_csv("./data/lawmaker_20_preprocessed.csv", index=False)
lawmaker_21_preprocessed.to_csv("./data/lawmaker_21_preprocessed.csv", index=False)

## 3. 국회의원을 node 리스트로 만들기


In [12]:
lawmaker_20_nodes = lawmaker_20_preprocessed
lawmaker_21_nodes = lawmaker_21_preprocessed
lawmaker_20_nodes.columns = [
    "Id",
    "Label",
    "Label_Hanja",
    "Birth",
    "Gender",
    "Elected",
    "Party",
    "Region",
]
lawmaker_21_nodes.columns = [
    "Id",
    "Label",
    "Label_Hanja",
    "Birth",
    "Gender",
    "Elected",
    "Party",
    "Region",
]

In [13]:
lawmaker_20_nodes.to_csv("./data/lawmaker_20_nodes.csv", index=False)
lawmaker_21_nodes.to_csv("./data/lawmaker_21_nodes.csv", index=False)

## 4. 공동 발의에 참여한 국회의원들 간의 관계를 나타내는 edge 리스트 만들기


In [14]:
pairs_20 = (
    laws_20_preprocessed["proposers"]
    .apply(lambda x: x if x is None else list(itertools.combinations(x, 2)))
    .explode()
)
pairs_20 = pd.DataFrame(pairs_20.tolist(), columns=["Source", "Target"])
pairs_20 = pairs_20[pairs_20["Target"].notnull()]
pairs_20.to_csv("./data/lawmaker_20_edges.csv", index=False)

In [ ]:
pairs_21 = (
    laws_20_preprocessed["proposers"]
    .apply(lambda x: x if x is None else list(itertools.combinations(x, 2)))
    .explode()
)
pairs_21 = pd.DataFrame(pairs_21.tolist(), columns=["Source", "Target"])
pairs_21 = pairs_21[pairs_21["Target"].notnull()]
pairs_21.to_csv("./data/lawmaker_21_edges.csv", index=False)